In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os, os.path, shutil
import zipfile
import tensorflow as tf
import h5py
import random
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications import *
from keras.preprocessing.image import *
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *
from os import walk



import warnings
warnings.filterwarnings('ignore')

In [ ]:
data_zip_dir = '/kaggle/input/dogs-vs-cats'
train_zip_dir = os.path.join(data_zip_dir, 'train.zip')
with zipfile.ZipFile(train_zip_dir, 'r') as z:
    z.extractall()


In [ ]:
train_dir = os.path.join(os.getcwd(), 'train')
test_dir = os.path.join(os.getcwd(), 'test1')
train_set_dir = os.path.join(train_dir, 'train')
os.mkdir(train_set_dir)
valid_set_dir = os.path.join(train_dir, 'valid')
os.mkdir(valid_set_dir)
test_set_dir = os.path.join(train_dir, 'test')
os.mkdir(test_set_dir)
dog_files = [f'dog.{i}.jpg' for i in range(12500)]
cat_files = [f'cat.{i}.jpg' for i in range(12500)]

In [ ]:
cd /kaggle/working/test1

In [ ]:
!ls

In [ ]:
for dog, cat in zip(dog_files[:10000], cat_files[:10000]):
    src = os.path.join(train_dir, dog)
    dst = os.path.join(train_set_dir, dog)
    shutil.move(src, dst)
    src = os.path.join(train_dir, cat)
    dst = os.path.join(train_set_dir, cat)
    shutil.move(src, dst)
for dog, cat in zip(dog_files[10000:11250], cat_files[10000:11250]):
    src = os.path.join(train_dir, dog)
    dst = os.path.join(valid_set_dir, dog)
    shutil.move(src, dst)
    src = os.path.join(train_dir, cat)
    dst = os.path.join(valid_set_dir, cat)
    shutil.move(src, dst)
for dog, cat in zip(dog_files[11250:12500], cat_files[11250:12500]):
    src = os.path.join(train_dir, dog)
    dst = os.path.join(test_set_dir, dog)
    shutil.move(src, dst)
    src = os.path.join(train_dir, cat)
    dst = os.path.join(test_set_dir, cat)
    shutil.move(src, dst)


In [ ]:
print(test_set_dir)

In [ ]:
print(f'the number of train set : {len(os.listdir(train_set_dir))}')
print(f'the number of valid set : {len(os.listdir(valid_set_dir))}')
print(f'the number of test set : {len(os.listdir(test_set_dir))}')

In [ ]:
folder_path = "/kaggle/working/train/train"

images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for image in images:
    folder_name = image.split('.')[0]

    new_path = os.path.join(folder_path, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(folder_path, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [ ]:
folder_path = "/kaggle/working/train/valid"

images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for image in images:
    folder_name = image.split('.')[0]

    new_path = os.path.join(folder_path, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(folder_path, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [ ]:
folder_path = "/kaggle/working/train/test"

images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

for image in images:
    folder_name = image.split('.')[0]

    new_path = os.path.join(folder_path, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(folder_path, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [ ]:
test_zip_dir = os.path.join(data_zip_dir, 'test1.zip')
with zipfile.ZipFile(test_zip_dir, 'r') as z:
    z.extractall("test_unzip")

In [ ]:
print(f"We have total {len(os.listdir('test_unzip/test1'))} images in our training data.")

In [ ]:
for (dirpath, dirnames, filenames) in walk("/kaggle/working/train/train"):
    print("Directory path: ", dirpath)
    print("Folder name: ", dirnames)
    print("File name: ", filenames)

In [ ]:
%time

BATCH_SIZE = 128
image_size = (128, 128)
EPOCHS = 200

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    # model = Model(base_model.input, GlobalMaxPooling2D()(base_model.output)), 결과값의 acc는 약간 0.1% 수준으로 낮지만, loss는 0.01(50%)수준으로 높아짐
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("/kaggle/working/train/train", image_size, shuffle=False, 
                                              batch_size=BATCH_SIZE)
    valid_generator = gen.flow_from_directory("/kaggle/working/train/valid", image_size, shuffle=False, 
                                             batch_size=BATCH_SIZE)
    test_generator = gen.flow_from_directory("/kaggle/working/train/test", image_size, shuffle=False, 
                                             batch_size=BATCH_SIZE, class_mode=None)
    real_test_generator = gen.flow_from_directory("test_unzip", image_size, shuffle=False, 
                                             batch_size=BATCH_SIZE, class_mode=None)

    train = model.predict_generator(train_generator, train_generator.samples)
    valid = model.predict_generator(valid_generator, valid_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    real_test = model.predict_generator(real_test_generator, real_test_generator.samples)

    with h5py.File("gap_%s.h5"%MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("valid", data=valid)
        h.create_dataset("test", data=test)
        h.create_dataset("real_test", data=real_test)
        h.create_dataset("train_label", data=train_generator.classes)
        h.create_dataset("valid_label", data=valid_generator.classes)

write_gap(ResNet50, (224, 224))
write_gap(Xception, (299, 299), xception.preprocess_input)
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)


In [ ]:
#y_train = np.array(h5py.File("gap_ResNet50.h5", 'r')['label'])
#y_train1 = np.array(h5py.File("gap_ResNet50.h5", 'r')['label'])
#y_train2 = np.array(h5py.File("gap_ResNet50.h5", 'r')['train'])
#y_train3 = np.array(h5py.File("gap_ResNet50.h5", 'r')['valid'])
#y_train4 = np.array(h5py.File("gap_ResNet50.h5", 'r')['test'])
#len(y_train1),len(y_train2),len(y_train3),len(y_train4)

In [ ]:
X_train = []
X_valid = []
X_test = []
X_real_test = []

for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        X_test.append(np.array(h['test']))
        X_real_test.append(np.array(h['real_test']))
        y_train = np.array(h['train_label'])
        y_valid = np.array(h['valid_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_real_test = np.concatenate(X_real_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)

In [ ]:
input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer=Adam(lr=0.0003, decay=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
def Callbacks():
    erl = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', 
                        restore_best_weights=True)
    rdc = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1, mode='min')
    return [erl,rdc]

In [ ]:
history = model.fit(X_train,
                    y_train, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=Callbacks(),
                    validation_data = (X_valid,y_valid)
                   )

In [ ]:
def plot_model_history(model_history, acc='accuracy', val_acc='val_accuracy'):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    axs[0].plot(range(1,len(model_history.history[acc])+1),model_history.history[acc])
    axs[0].plot(range(1,len(model_history.history[val_acc])+1),model_history.history[val_acc])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history[acc])+1),len(model_history.history[acc])/10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()
    
plot_model_history(history)

In [ ]:
#test_generator = gen.flow_from_directory("/kaggle/working/test1", image_size, shuffle=False, 
#                                     batch_size=BATCH_SIZE, class_mode=None)

#X_test = model.predict_generator(test_generator, test_generator.samples)
#h5py.File("gap_%s.h5"%MODEL.__name__) as h:
#        h.create_dataset("train", data=train)
#        h.create_dataset("valid", data=valid)
#        h.create_dataset("test", data=test)
#        h.create_dataset("train_label", data=train_generator.classes)
#        h.create_dataset("valid_label", data=valid_generator.classes)

In [ ]:
y_pred = model.predict(X_real_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
len(y_pred)

In [ ]:
test_filenames = os.listdir('test_unzip/test1')
df = pd.DataFrame({'filename': test_filenames})
df.head()
df.shape

In [ ]:
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test_unzip/", image_size, shuffle=False, 
                                         batch_size=BATCH_SIZE, class_mode=None)

In [ ]:
df['category'] = y_pred
df.head()

In [ ]:
threshold = 0.5
df['category'] = np.where(y_pred > threshold, "Cat","Dog")



df.to_csv('submission.csv', index=False)
df.head()

In [ ]:
df['category'].value_counts().plot.bar()


In [ ]:
sample_test = df.sample(n=36).reset_index()
sample_test.head()

plt.figure(figsize=(24, 16))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("/kaggle/working/test_unzip/test1/"+filename, target_size=image_size)
    plt.subplot(6, 6, index+1)
    plt.imshow(img)
    plt.xlabel('It\'s a ' + "{}".format(category) )
plt.tight_layout()
plt.show()